# Global value chains: a networks approach
### Katrina Walker, Frank Fanteev, David Rosenfeld

## Introduction

An increasingly important feature of global trade and economics since the 1980s has been the breaking up of production processes into what is known as Global Value Chains (GVCs), in part due to increased ease of communication and coordination between different locations enabled by new telecommunication and internet technologies (Baldwin, 2016). The increasing global interconnections generated by that process has has also increased the vulnerability of the global economic system to contagion of economic shocks. 

As such, understanding the nature of economic interconnections within these global value chains has become a priority area of research for governments and policy research institutions. In particular, the OECD has developed Inter-Country Input-Output (ICIO) tables, and indicators of Trade in Value Added (TiVA) which have improved our understanding of GVCs.

We will use the OECD's ICIO data in a network setting to analyse the GVCs, and specifically use community-detection algorithms to analyse the extent to which GVCs have evolved between 1995 and 2011.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Import the input-output table, with multi-indices for both index and columns: one for country, another for sector
input_output = pd.read_csv("input_output.csv", index_col = [0,1], header = [0,1])